In [42]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.retrievers import WikipediaRetriever
import json


def get_duckduckgo_data(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    query = inputs["query"]
    return ddg.run(query)


def get_wikipedia_data(inputs):
    retriever = WikipediaRetriever(top_k_results=2)
    query = inputs["query"]
    docs = retriever.get_relevant_documents(query)
    docs = "\n\n".join(document.page_content for document in docs)
    return docs


functions_map = {
    "get_duckduckgo_data": get_duckduckgo_data,
    "get_wikipedia_data": get_wikipedia_data,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_duckduckgo_data",
            "description": "From given query, find a main keyword and get data from duckduckgo.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Searching query which contains a main keyword.",
                    },
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_wikipedia_data",
            "description": "From given query, find a main keyword and get data from wikipedia.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Searching query which contains a main keyword.",
                    },
                },
                "required": ["query"],
            },
        },
    },
]

In [13]:
from openai import OpenAI

client = OpenAI()

# assistant = client.beta.assistants.create(
#     name = "Investor Assistant",
#     instructions="""You are a very meticulous Data searcher.

#                 You want to find information from every source possible.

#                 You can find information from both DuckDuckGo and Wikipedia.

#                 After searching data from duckduckgo and wikipedia, You should write a professional report about main keyword.""",
#     model = "gpt-3.5-turbo",
#     tools=functions,
# )

assistant_id = "asst_K9059MPIX3nrBVZRGHubVTEA"

In [36]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content" : "What is a blue archive?"
        }
    ]
)

thread

Thread(id='thread_5Mf6piqe0XTwo9uHCi6bANgu', created_at=1712902655, metadata={}, object='thread')

In [37]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

run

Run(id='run_ZylJYPaMFIpSArhZS6jEHW2O', assistant_id='asst_K9059MPIX3nrBVZRGHubVTEA', cancelled_at=None, completed_at=None, created_at=1712902658, expires_at=1712903258, failed_at=None, file_ids=[], instructions='You are a very meticulous Data searcher. You should find information from both duckduckgo and wikipedia.            You can find information from both DuckDuckGo and Wikipedia.\nAfter searching data from duckduckgo and wikipedia, You should write a professional report about main keyword.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_5Mf6piqe0XTwo9uHCi6bANgu', tools=[FunctionTool(function=FunctionDefinition(name='get_duckduckgo_data', description='From given query, find a main keyword and get data from duckduckgo.', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Searching query which contains a main keyword.'}}, 'required': ['query']})

In [43]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id
    ).data
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def get_tools_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        outputs.append({
            "tool_call_id" : action_id,
            "output": functions_map[function.name](json.loads(function.arguments))
        })
    return outputs


def submit_tools_outputs(run_id, thread_id):
    outputs = get_tools_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id = run_id,
        thread_id=thread_id,
        tool_outputs=outputs,
    )




In [38]:
get_run(run.id, thread.id).status

'requires_action'

In [39]:
get_messages(thread.id)

user: What is a blue archive?


In [44]:
submit_tools_outputs(run.id, thread.id)

Calling function: get_duckduckgo_data with arg {"query": "Blue Archive"}
Calling function: get_wikipedia_data with arg {"query": "Blue Archive"}


c:\Users\CWS\Documents\NomadCoder\python_gpt\env\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


[{'tool_call_id': 'call_grqi8gbO2oZxMnmgiXAaVFOs',
  'output': 'MALxJapan -More than just anime-. [⏰Time-limited] Try this manga—for free! Looking for information on the anime Blue Archive the Animation? Find out more with MyAnimeList, the world\'s most active online anime and manga community and database. The city\'s academies are divided into their own districts and are considered mostly independent. Trip Trap Train Overview. Events. senea. April 1, 2024. Table of Contents [ Show] Following the release of the previous Millennium rerun events, we are introduced to a new event " Trip Trap Train ". This event features 2 new Sonic students Ichika and Kasumi, who will be added to the permanent pool of students once their banner ends. An overview of all characters implemented on the wiki. Please also see the following listings: Interactive chart of character stats; Unique weapons list Learn how to rank and summon characters in Blue Archive, a gacha game based on the Federal Investigation C